# Importing Necessary Library

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import joblib,os

In [ ]:
cwd = os.getcwd()
os.makedirs("data",exist_ok=True)
data_path = os.path.join(cwd,"data","iot_synthetic_sensor_data.csv")
# Load dataset
df = pd.read_csv(data_path)

# Encode target labels with custom order
custom_order = ["Safe", "Warning", "Gas Leak", "Fire Detected", "Evacuate Immediately"]
le = LabelEncoder()
le.fit(custom_order)
df["label_encoded"] = le.transform(df["label"])  # Using 'label' as confirmed

# Separate features and target
features = ['temperature', 'humidity', 'mq2_smoke', 'mq135_gas', 'flame_detected', 'cv_flame_score', 'cv_smoke_score', 'person_detected']
X = df[features]
y = df["label_encoded"]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Scale numeric features
numeric_features = ['temperature', 'humidity', 'mq2_smoke', 'mq135_gas', 'flame_detected', 'cv_flame_score', 'cv_smoke_score', 'person_detected']
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train[numeric_features])
X_test_scaled = scaler.transform(X_test[numeric_features])

# Convert scaled arrays back to DataFrame to maintain compatibility
X_train = pd.DataFrame(X_train_scaled, columns=numeric_features, index=X_train.index)
X_test = pd.DataFrame(X_test_scaled, columns=numeric_features, index=X_test.index)

# Initialize Random Forest
rf_clf = RandomForestClassifier(
    n_estimators=100,
    max_depth=None,
    random_state=42,
    n_jobs=-1,
    class_weight="balanced"
)

# Train the model
rf_clf.fit(X_train, y_train)

# Predict and evaluate
y_pred = rf_clf.predict(X_test)
print("\n✅ Classification Report:")
print(classification_report(y_pred, y_test, target_names=le.classes_))

# Save the model and preprocessing objects
joblib.dump(rf_clf, r'threat_model.pkl')
joblib.dump(le, r'label_encoder.pkl')
joblib.dump(scaler, r'scaler.pkl')
print("Random Forest model, label encoder, and scaler saved to backend/.")